# Downloading executable to Target
The target needs to be connected to the PC via a debug adapter. This is also known as ICSP - In-Circuit-Serial-Programmer

The target understands SWD or JTAG protocols, the PC understands UBS protocol. So, there needs to be a translator in between that can convert USB to SWD/JTAG.

On the host side, we need to run some program to talk to the debug adapter. One such program in sthe  [_OpenOCD_](https://openocd.org/). 

## OpenOCD
 - Open source
 - Supports various architectures ARM, Cortex ...
 - Flash programming support
 - Programming adapters do protocol conversion
 - Some popular debug adapters are Segger J-Link, Ulink, ST-Link
 - Some evaluation boards arenow coming with their build-in debug adapters
### Workflow
- Host PC opens an OpenOCD session
- Open OCD uses ST-Link driver (needs to be installed) in the computer and will talk to the debug adapter
- If you use SWD, you will just use SWand talk to the microcontroller DCLK and SWDIO. 
- The microcontroller provides two pins to accept the firmware update
- Inside the MCU, you have a debug access port, which knows how to talk to the processor
- By using the debug access port, you can access various buses and access points
- In case of flashing, we want to talk to the flash memory, which happens thru the AHB-AP (Ap- Access Point)
- AHB-AP also helps with debugging. 
- When USB data packets are sent thru the host, they get converted to SWD packets by the debug adapter. And the AHB-AP routes packets to flash memory. 

## Steps to download the code using OpenOCD
1. Download and install OpenOCD
2. Install Telnet client
3. Run OpenOCD with the board configuration file
4. Issue commands over telnet and GDB client to download and debug the code

### Step 3: Running OpenOCD
You can directly write the command to flash the target in the `Makefile`. 
In the `Makefile`, just add another recipe
```
load:
	openocd -f board/st_nucleo_f4.cfg
```

For a different board, you will need to add a different `.cfg` file. These are available inside `/usr/share/openocd/scripts/board/` path. Just choose the correct processor config file. 

Now connect the target and run `make load` on the terminal

![image](images/openOCD_Session_Start.png)

This will show that the open OCD session is running correctly. Now keep this window running in the background and open another tab. We will start step 4 to kick off a telnet client. 

### Step 4: Issuing commands over Telnet client

1. `gdb-multiarch`
2. `target remote localhost:3333`
3. `monitor reset init`
4. `monitor flash write_image final.elf` - insert your elf file of choice
5. `monitor reset halt`
6. `monitor reset resume`
7. `disconnect` - ends connection
8. `quit` - exits gdb shell

`ctrl+c` in OpenOCD terminal exits from the session and the MCU is now on its own and programmed properly. 
There you go!


If step 4 works correctly, you'll see this in the gdb window:
![image](images/gdb_successful_flash_image.png)

> Flash commands GDB documentation: https://openocd.org/doc/html/Flash-Commands.html#Flash-Configuration-Commands

> 
